In [1]:
import numpy as np
import json, requests
import pandas as pd
import random
import cmlapi
import os

### Overview

the notebook will:
1. take input format for python endpoint
2. extract model url and model access key
3. reading input data, will take a single sample of link/location combination 
4. then take a random sample of 25 contiguous rows
5. Convert this into json format through 'dataframe_to_json' function
6. Put all pieces together for json payload required by endpoint
7. Send prediction request
8. 'Unpack' response

In [2]:
window = 24

In [3]:
def dataframe_to_json(df):
    """Create a dictionary that will later be converted to a JSON object
    ensure that 
    """
    data = {
        'rx_gbs': df['rx_gbs'].iloc[-(window+1):].tolist(),
        'tx_gbs': df['tx_gbs'].iloc[-(window+1):].tolist(),
        'time': [df['time'].iloc[-1]],  # Only the last time entry
        'link': [df['link'].iloc[-1]],  # Only the last link entry
        'location': [df['location'].iloc[-1]]  # Only the last location entry
    }
    
    return data

In [4]:
df = pd.read_csv('data/netstats_hourly_4_3.csv')
#df.head()

In [7]:
model_name = input("Enter the name of the model: ")

Enter the name of the model:  LSTM-no-metrics


In [43]:
## Need to get the below prgramatically in the future iterations
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))
model_key = target_model.models[0].access_key

## Save the access key for the model to the environment variable of this project

# model_url = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model?accessKey=")
# model_url = model_url + model_key
model_url = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model")

In [44]:
model_url

'https://modelservice.ml-cd558cf0-ec7.se-sandb.a465-9q4k.cloudera.site/model'

In [45]:
model_key

'm5nezecapjhr5vhypkyfk7ow3p8kdanz'

Pick a random link / location

In [34]:
sample_df = df[['link','location']].drop_duplicates().sample()
link_pick = sample_df['link'].values[0]
location_pick = sample_df['location'].values[0]
print(sample_df)

     link location
2188  ER1      AMS


In [35]:
# test_df has only one combintation  of link_pick and location_pick
test_df = df[(df.link == link_pick) & (df.location == location_pick)]

In [36]:
# pick a random segment of size window + 1
start_row = np.random.randint(0, len(test_df) - (window+1))
random_segment = df.iloc[start_row:start_row + (window+1)]

In [37]:
# confirm size
random_segment.shape

(25, 6)

In [38]:
# convert input to json
test_inputs = dataframe_to_json(random_segment)

In [39]:
# # registry specific parameters
# model_url = 'https://modelservice.cml.apps.puncdsocpprd.os.net.ibm.com/model'
# model_key = "m0k1lk3c7aae8k16tniqtddopp77yz0l"

Step below puts it into required payload for endpoint

In [41]:
#r = requests.post(MODEL_ENDPOINT, data=json.dumps(test_inputs), headers={'Content-Type': 'application/json'})

In [42]:
#r.json()

{'success': False,
 'ReplicaID': 'lstm-no-metrics-10-21-546d45d488-8bbhq',
 'Size': 79,
 'StatusCode': 400}

In [46]:
# build embedded dictionary step 1
request_dict = {"request":test_inputs}

# access key will be end point specific
BackDict = {"accessKey":model_key}
BackDict.update(request_dict)
request_dict=BackDict

In [47]:
r = requests.post(model_url, data=json.dumps(request_dict), headers={'Content-Type': 'application/json'})

In [48]:
r.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:af1ca1b6-2438-44e1-910f-e06f3320a279/0a76967a-59b2-4390-a481-f5998560b746',
  'prediction': {'rx_bytes': [[9.300664901733398,
     8.624772071838379,
     7.315581321716309,
     6.047887325286865,
     5.334909439086914,
     4.521298885345459]],
   'time': ['2024-04-05T16:00:00',
    '2024-04-05T17:00:00',
    '2024-04-05T18:00:00',
    '2024-04-05T19:00:00',
    '2024-04-05T20:00:00',
    '2024-04-05T21:00:00'],
   'tx_bytes': [[0.4562716782093048,
     -0.4851105213165283,
     -1.0172359943389893,
     -0.9375834465026855,
     -0.73171466588974,
     -1.0454868078231812]]},
  'uuid': 'f4ef630a-54c3-4c75-9cab-a6cc951c303a'},
 'ReplicaID': 'lstm-no-metrics-10-21-546d45d488-8bbhq',
 'Size': 653,
 'StatusCode': 200}

Get the model response

In [49]:
r.json()['response']['prediction']

{'rx_bytes': [[9.300664901733398,
   8.624772071838379,
   7.315581321716309,
   6.047887325286865,
   5.334909439086914,
   4.521298885345459]],
 'time': ['2024-04-05T16:00:00',
  '2024-04-05T17:00:00',
  '2024-04-05T18:00:00',
  '2024-04-05T19:00:00',
  '2024-04-05T20:00:00',
  '2024-04-05T21:00:00'],
 'tx_bytes': [[0.4562716782093048,
   -0.4851105213165283,
   -1.0172359943389893,
   -0.9375834465026855,
   -0.73171466588974,
   -1.0454868078231812]]}